# Split Titanic data upfront - A simple attempt for separate predictions


---

The Age-feature in the Titanic data is not defined for many passengers. Several good Kaggle notebooks do feature engineering to come up with replacements of the missing age data (e.g. [Titanic Data Science Solutions](https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook)).

I was wondering if it would be possible to just .... 
1. ** separate the training and test data ** into two buckets (Age defined vs. Age NaN), 
2. do **predictions separately**, and 
3. finally, ** concatenate the results**.


Additionally, the notebook includes basic hyperparameter tuning for RandomForest and AdaBoost.

## Overview:

- Import modules, read data and display some data
- Prepare subset of training and test data without any NaN in Age
- Predict 'Survived' for passengers data with Age
 - Grid search and fit with Random Forest Classifier
 - Grid search and fit with AdaBoost Classifier
 - Predict based on better algorithm and score
- Predict 'Survived' with Random Forest Classifier for passengers data without Age
- Concatenate predictions
- Summary



### Import modules, read data and display some data


As usual, import modules, read data and display some data

In [ ]:
# data processing
import numpy as np
import pandas as pd 


import warnings
warnings.filterwarnings('ignore')

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold



# some configuratin flags and variables
verbose = 1 # Use in classifier
quick_run = True # if set to True, use only few variabales during hyperparameter tuning
n_jobs = -1

# Input files
file_train='../input/train.csv'
file_test='../input/test.csv'

# define random seed for reproducibility
seed = 69
np.random.seed(seed)

# read training and test data
train_df = pd.read_csv(file_train,index_col='PassengerId')
test_df = pd.read_csv(file_test,index_col='PassengerId')


#### Brief data preview:

In [ ]:
# Show the columns
train_df.columns.values

In [ ]:
# Show the shape
train_df.shape

In [ ]:
# preview the training data
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
# Show that there is NaN data (Age,Fare Embarked), that needs to be handled.
train_df.isnull().sum()

#### A function for simple data cleansing without the Age features
- Drop unwanted features ['Name', 'Ticket', 'Cabin']
- Fill missing data: Fare with the mean, Embarked with most frequent value
- Convert categorical features into numeric
- Convert Embarked to one-hot
- Note, missing data of the Age features is not filled.

In [ ]:
def prep_data(df):
    # Drop unwanted features
    df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    
    # Fill missing data:  Fare with the mean, Embarked with most frequent value
    df[['Fare']] = df[['Fare']].fillna(value=df[['Fare']].mean())
    df[['Embarked']] = df[['Embarked']].fillna(value=df['Embarked'].value_counts().idxmax())
    
    # Convert categorical  features into numeric
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
      
    # Convert Embarked to one-hot
    enbarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(enbarked_one_hot)

    return df


### Prepare subset of training and test data without any NaN in Age

Prepare training data without any NaN in Age (**train_df_age_no_nan**). 
Show that there isn't any null data

In [ ]:
train_df = prep_data(train_df)
train_df_age_no_nan = train_df[train_df['Age'].notnull()]
print("Training data without any NaN in Age:\n", train_df_age_no_nan.isnull().sum())

In [ ]:
train_df_age_no_nan.shape

**Visualise feature correlation with a heatmap**

As expected, only the Gender is correlated to 'Survived'

In [ ]:
cmap = plt.cm.RdBu
corr = train_df_age_no_nan.corr()
plt.figure(figsize=(12,10))
plt.title('Pearson Features Correlation of training data without NaN in Age', size=15)
sns.heatmap(corr, cmap=cmap,  annot=True, linewidths=1)

** Show pairplots to visualize the distribution of data across features **

Further drill down could be helpfull. There are several Kaggel notebooks showing in-depth visualisations, data exploration and wrangling on the Titanic data:
- [In-Depth Visualisations - Simple Methods](https://www.kaggle.com/jkokatjuhha/in-depth-visualisations-simple-methods)
- [Titanic Data Exploration Starter](https://www.kaggle.com/neviadomski/titanic-data-exploration-starter)
- [Titanic Data Science Solutions](https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook)


In [ ]:
g = sns.pairplot(train_df_age_no_nan, hue='Survived', palette = 'seismic',size=1.5,diag_kind='kde',diag_kws=dict(shade=True),plot_kws=dict(s=10))
g.set(xticklabels=[])

Prepare test data without any NaN in Age (**test_df_age_no_nan**). 

In [ ]:
test_df = prep_data(test_df)
test_df_age_no_nan = test_df[test_df['Age'].notnull()]
print("Test data without any NaN in Age:\n", test_df_age_no_nan.isnull().sum())

In [ ]:
test_df_age_no_nan.shape

### Predict 'Survived' for passengers data with Age

Split training data into input X and output Y

In [ ]:
# X contains all columns except 'Survived'  
X = train_df_age_no_nan.drop(['Survived'], axis=1).values.astype(float)

# Y is just the 'Survived' column
Y = train_df_age_no_nan['Survived'].values

#### Grid search and fit with Random Forest Classifier

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)

if quick_run:
    rf_parameters = {"max_depth": [4,8]
                ,"min_samples_split" :[2,6]
                ,"n_estimators" : [100]
                ,"min_samples_leaf": [1,2]
                ,"max_features": [6,"sqrt"]
                ,"criterion": ['gini']}       
else:
    rf_parameters = {"max_depth": [2,4,6,8,12]
                ,"min_samples_split" :[2,3,5,8]
                ,"n_estimators" : [50, 100,200]
                ,"min_samples_leaf": [2,3,5]
                ,"max_features": [4,6,"sqrt"]
                ,"criterion": ['gini','entropy']}

print('** GridSearchCV RF ...') 
rf_clf = RandomForestClassifier()
rf_grid = GridSearchCV(rf_clf,rf_parameters, n_jobs = n_jobs, verbose = verbose, cv = 10)
rf_grid.fit(X,Y)

print('Best score RF: {}'.format(rf_grid.best_score_))
print('Best parameters RF: {}'.format(rf_grid.best_params_))

rf_clf = RandomForestClassifier(**rf_grid.best_params_)
rf_clf.fit(X,Y)

#### Grid search and fit with AdaBoost Classifier

In [ ]:
if quick_run:
    ab_parameters = {'n_estimators':[50,100,200,300],
                  'learning_rate':[0.1,0.5,1.0,2.0]}
else:
    ab_parameters = {'n_estimators':[50,100],
                  'learning_rate':[0.5,1.0]}
    
print('** GridSearchCV AB ...') 
ab_clf = AdaBoostClassifier()
ab_grid = GridSearchCV(ab_clf,ab_parameters, n_jobs = n_jobs, verbose = verbose, cv = 10)
ab_grid.fit(X,Y)

print('Best score AB: {}'.format(ab_grid.best_score_))
print('Best parameters AB: {}'.format(ab_grid.best_params_))

ab_clf = AdaBoostClassifier(**ab_grid.best_params_)
ab_clf.fit(X,Y)

#### Predict based on better algorithm and score

In [ ]:
# Create X_test
X_test = test_df_age_no_nan.values.astype(float)


# Predict 'Survived' for Age no NaN
if ( rf_grid.best_score_ > ab_grid.best_score_ ) :
    print('** Predict Survived for data with age using RF {}'.format(rf_grid.best_params_))
    prediction_age_no_nan = rf_clf.predict(X_test)
else:
    print('** Predict Survived for data with age using AB {}'.format(ab_grid.best_params_))
    prediction_age_no_nan = ab_clf.predict(X_test)


subm_no_nan = pd.DataFrame({
    'PassengerId': test_df_age_no_nan.index,
    'Survived': prediction_age_no_nan,
})
    
test_df_age_nan = test_df[test_df['Age'].isnull()]

### Predict 'Survived' for passengers data without Age

#### Grid search,  fit and predict with Random Forest Classifier

In [ ]:
# Split training data into input X and output Y

train_df_age_nan = train_df[train_df['Age'].isnull()]

# X contains all columns except Age and 'Survived'  
X = train_df.drop(['Age','Survived'], axis=1).values.astype(float)

# Y is just the 'Survived' column
Y = train_df['Survived'].values


kfold = KFold(n_splits=10, random_state=seed)

print('** GridSearchCV (no Age) RF ...') 
rf_clf = RandomForestClassifier()
rf_grid = GridSearchCV(rf_clf,rf_parameters, n_jobs = n_jobs, verbose = verbose, cv = 10)
rf_grid.fit(X,Y)

print('Best score (no Age) RF: {}'.format(rf_grid.best_score_))
print('Best parameters (no Age) RF: {}'.format(rf_grid.best_params_))


rf_clf = RandomForestClassifier(**rf_grid.best_params_)
rf_clf.fit(X,Y)

# Predict test data with Age == NaN
X_test_age_nan = test_df_age_nan.drop(['Age'], axis=1).values.astype(float)

# Predict 'Survived'
prediction_age_nan = rf_grid.predict(X_test_age_nan)   

subm_nan = pd.DataFrame({
    'PassengerId': test_df_age_nan.index,
    'Survived': prediction_age_nan
})

### Concatenate predictions

In [ ]:
# stack the DataFrames on top of each other
submission = pd.concat([subm_no_nan, subm_nan], axis=0)

submission.sort_values('PassengerId', inplace=True)    
submission.to_csv('submission-splitt-input.csv', index=False)

## Summary
The score is not really great (~0.76) but comparable to many other simple approaches. My experiment illustrated that splitting the test and training data upfront due to data quality issues could be a reasonable approach.


Your feedback and suggestions are very welcome.